viendo el poco exito de los modelos entrenados procederemos a usar las imagenes y la metadata a la vez.

La clave aquí es vincular los datos tabulares con las imágenes utilizando el índice (que coincide con el nombre de los archivos de imagen) para combinar la información.  implementaremos un modelo multimodal que use tanto imágenes como datos tabulares.

In [34]:
#cargamos librrias necesarias
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score

In [50]:


# Cargar el dataset y asegurarse de que el índice se conserve
df = pd.read_csv('/Users/luiseduardogarciablanco/Desktop/bootcamp/Proyecto_Final_Cancer/src/metadata_processed_ntk.csv', index_col='isic_id')

# Separar características y objetivo
X = df.drop(columns=['target'])
y = df['target']

# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el tamaño de la imagen
img_size = (224, 224)
batch_size = 32

df

,target,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,...,tbp_lv_location_Torso_Back,tbp_lv_location_Torso_Front,tbp_lv_location_Upper,tbp_lv_location_simple_Left_Arm,tbp_lv_location_simple_Left_Leg,tbp_lv_location_simple_Right_Arm,tbp_lv_location_simple_Right_Leg,tbp_lv_location_simple_Torso_Back,tbp_lv_location_simple_Torso_Front,iddx_1_Indeterminate
isic_id,,,,,,,,,,,,,,,,,,,,,
ISIC_0015670,0,60.0,3.04,20.244422,16.261975,26.922447,23.954773,33.684638,28.953117,53.058545,...,0,0,1,0,0,0,1,0,0,0
ISIC_0015845,0,60.0,1.10,31.712570,25.364740,26.331000,24.549290,41.219030,35.299260,39.702910,...,0,0,0,0,0,0,0,0,0,0
ISIC_0015864,0,60.0,3.40,22.575830,17.128170,37.970460,33.485410,44.174920,37.611800,59.265850,...,0,0,0,0,0,0,0,1,0,0
ISIC_0015902,0,65.0,3.22,14.242329,12.164757,21.448144,21.121356,25.746200,24.374023,56.414429,...,0,0,0,0,0,0,0,0,1,0
ISIC_0024200,0,55.0,2.73,24.725520,20.057470,26.464900,25.710460,36.217980,32.608740,46.946070,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ISIC_9999937,0,70.0,6.80,22.574335,14.944666,27.663259,26.767135,35.705132,30.656525,50.784168,...,0,0,0,0,0,0,0,0,1,0
ISIC_9999951,0,60.0,3.11,19.977640,16.026870,34.158840,31.011870,39.571870,34.908400,59.678970,...,0,0,0,0,0,0,0,1,0,0
ISIC_9999960,0,65.0,2.05,17.332567,12.364397,29.845326,26.500073,34.513206,29.242644,59.854275,...,0,0,0,0,0,0,0,0,1,0


In [47]:

#Define el generador de datos para cargar imágenes y datos tabulares por lotes.
def load_image(img_name, img_size):
    img_path = os.path.join('/Users/luiseduardogarciablanco/Desktop/bootcamp/Archivos_proyecto_cancer/isic-2024-challenge/Filtered_images', img_name + '.jpg')
    if not os.path.exists(img_path):
        print(f"Advertencia: El archivo {img_path} no se encuentra.")
        return np.zeros((img_size[0], img_size[1], 3))
    img = load_img(img_path, target_size=img_size)
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

def data_generator(X_tabular, y, batch_size, img_size):
    num_samples = len(X_tabular)
    while True:
        for offset in range(0, num_samples, batch_size):
            end = min(offset + batch_size, num_samples)
            X_batch_tabular = X_tabular.iloc[offset:end].values
            y_batch = y.iloc[offset:end].values
            image_names_batch = X_tabular.index[offset:end]
            images = np.array([load_image(img_name, img_size) for img_name in image_names_batch])
            yield [images, X_batch_tabular], y_batch

In [52]:
#Define y compila el modelo que combina imágenes y datos tabulares.

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

def build_model(input_shape_tabular, img_size):
    # Cargar un modelo preentrenado (ResNet50) para la parte de imágenes
    resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(*img_size, 3))
    for layer in resnet.layers:
        layer.trainable = False

    # Crear la rama para procesar las imágenes
    image_input = Input(shape=(*img_size, 3))
    x = resnet(image_input)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)

    # Crear la rama para procesar los datos tabulares
    tabular_input = Input(shape=(input_shape_tabular,))
    y = Dense(128, activation='relu')(tabular_input)
    y = Dropout(0.3)(y)
    y = Dense(64, activation='relu')(y)

    # Fusionar ambas ramas
    combined = concatenate([x, y])
    z = Dense(64, activation='relu')(combined)
    z = Dropout(0.3)(z)
    output = Dense(1, activation='sigmoid')(z)

    # Definir el modelo completo
    model = Model(inputs=[image_input, tabular_input], outputs=output)

    # Compilar el modelo
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    # Compilar el modelo con el optimizador legado
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


    return model

Aquí utilizamos ResNet50 preentrenado para la parte de imágenes y lo combinamos con un modelo para los datos tabulares.

Integrar el uso de un modelo preentrenado (en este caso, ResNet50) en el flujo de trabajo con datos tabulares e imágenes. Este enfoque puede mejorar los resultados al aprovechar las capacidades de un modelo profundo que ya ha sido entrenado en una gran cantidad de datos de imágenes.

In [54]:
from tensorflow.keras.callbacks import EarlyStopping

batch_size = 32
img_size = (224, 224)  # Tamaño de las imágenes

# Crear generadores de datos
train_generator = data_generator(X_train, y_train, batch_size, img_size)
validation_generator = data_generator(X_val, y_val, batch_size, img_size)

# Calcular pasos por época y validación
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

# Construir el modelo
model = build_model(X_train.shape[1], img_size)

# Entrenar el modelo
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3)]
)


Epoch 1/5


2024-08-19 17:02:39.254324: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9547/9547 [==============================] - ETA: 0s - loss: 4.3384 - accuracy: 0.9979

2024-08-19 17:28:20.485878: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9547/9547 [==============================] - 1918s 201ms/step - loss: 4.3384 - accuracy: 0.9979 - val_loss: 0.5146 - val_accuracy: 0.9981
Epoch 2/5
9547/9547 [==============================] - 2083s 218ms/step - loss: 3.6111 - accuracy: 0.9980 - val_loss: 0.9247 - val_accuracy: 0.9989
Epoch 3/5
9547/9547 [==============================] - 2083s 218ms/step - loss: 4.3681 - accuracy: 0.9979 - val_loss: 6.5956 - val_accuracy: 0.9990
Epoch 4/5
9547/9547 [==============================] - 2082s 218ms/step - loss: 4.6700 - accuracy: 0.9980 - val_loss: 1.1903 - val_accuracy: 0.9990


In [55]:
from sklearn.metrics import recall_score, f1_score

def evaluate_model(model, generator, steps):
    y_true = []
    y_pred = []
    for _ in range(steps):
        (images, tabular_data), labels = next(generator)
        predictions = model.predict([images, tabular_data])
        y_true.extend(labels)
        y_pred.extend(predictions.flatten())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred) > 0.5  # Convertir probabilidades en etiquetas

    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return recall, f1

# Evaluar el modelo
recall, f1 = evaluate_model(model, validation_generator, validation_steps)

print(f"Recall: {recall}")
print(f"F1-score: {f1}")

2024-08-19 19:20:04.244591: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 31ms/step


2024-08-19 19:29:01.142023: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 29ms/step
Recall: 0.012987012987012988
F1-score: 0.025641025641025647


Recall: 0.012987012987012988
F1-score: 0.025641025641025647

no mejora con las imagenes y dataset juntos al entrenar un modelo de DL, toca buscra mas datos en datasets de otros años